In [168]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

def topoplot(datavector, chan_locs, **kwargs):
    # Set default values
    noplot = kwargs.get('noplot', 'off')
    plotgrid = kwargs.get('plotgrid', 'off')
    plotchans = kwargs.get('plotchans', [])
    handle = None
    Zi = None
    chanval = np.nan
    rmax = 0.5  # actual head radius
    INTERPLIMITS = 'head'
    INTSQUARE = 'on'
    default_intrad = 1
    ELECTRODES = kwargs.get('ELECTRODES', 'on')
    MAXDEFAULTSHOWLOCS = 64
    intrad = kwargs.get('intrad', np.nan)
    plotrad = kwargs.get('plotrad', np.nan)
    headrad = kwargs.get('headrad', 0.5)
    squeezefac = 1.0
    ContourVals = datavector

    # Handle additional arguments
    if 'noplot' in kwargs:
        noplot = kwargs['noplot']
        if not isinstance(noplot, str):
            if len(noplot) != 2:
                raise ValueError("'noplot' location should be [radius, angle]")
            else:
                chanrad = noplot[0]
                chantheta = noplot[1]
                noplot = 'on'

    # Set colormap
    cmap = plt.get_cmap('jet')
    cmaplen = cmap.N
    GRID_SCALE = 32

    if len(datavector) > MAXDEFAULTSHOWLOCS:
        ELECTRODES = 'off'
    else:
        ELECTRODES = 'on'

    datavector = np.array(datavector).flatten()
    ContourVals = np.array(ContourVals).flatten()

    # Read the channel location information
    tmpeloc = chan_locs
    labels = [loc['labels'] for loc in tmpeloc]
    indices = [i for i, loc in enumerate(tmpeloc) if 'theta' in loc]
    Th = np.array([tmpeloc[i]['theta'] if i in indices and not isinstance(tmpeloc[i]['theta'], np.ndarray) else np.nan for i in range(len(tmpeloc))])
    Rd = np.array([tmpeloc[i]['theta'] if i in indices and not isinstance(tmpeloc[i]['radius'], np.ndarray) else np.nan for i in range(len(tmpeloc))])
    Th = np.deg2rad(Th)
    allchansind = list(range(len(Th)))
    plotchans = indices

    if len(tmpeloc) == len(datavector) + 1:
        if plotchans and plotchans[-1] == len(tmpeloc):
            plotchans.pop()

    if len(datavector) > 1:
        inds = np.union1d(np.where(np.isnan(datavector))[0], np.where(np.isinf(datavector))[0])
        plotchans = list(set(plotchans) - set(inds))

    x, y = np.cos(Th) * Rd, np.sin(Th) * Rd
    plotchans = np.abs(plotchans)
    allchansind = np.array(allchansind)[plotchans]
    Th, Rd, x, y = Th[plotchans], Rd[plotchans], x[plotchans], y[plotchans]
    labels = np.array(labels)[plotchans]

    if np.isnan(plotrad):
        plotrad = min(1.0, max(Rd) * 1.02)
        plotrad = max(plotrad, 0.5)

    if np.isnan(intrad):
        default_intrad = 1
        intrad = min(1.0, max(Rd) * 1.02)
    else:
        default_intrad = 0
        if plotrad > intrad:
            plotrad = intrad

    pltchans = np.where(Rd <= plotrad)[0]
    intchans = np.where((x <= intrad) & (y <= intrad) if INTSQUARE == 'on' else (Rd <= intrad))[0]

    labels = labels[pltchans]

    if datavector is not None and len(datavector) > 1:
        intdatavector = datavector[intchans]
        datavector = datavector[pltchans]
        ContourVals = ContourVals[pltchans]

    squeezefac = rmax / plotrad
    Rd, intRd = Rd * squeezefac, Rd[intchans] * squeezefac
    x, y, intx, inty = x[pltchans] * squeezefac, y[pltchans] * squeezefac, x[intchans] * squeezefac, y[intchans] * squeezefac

    if default_intrad:
        xmin, xmax = min(-rmax, min(intx)), max(rmax, max(intx))
        ymin, ymax = min(-rmax, min(inty)), max(rmax, max(inty))
    else:
        xmin, xmax, ymin, ymax = -intrad * squeezefac, intrad * squeezefac, -intrad * squeezefac, intrad * squeezefac

    xi, yi = np.linspace(xmin, xmax, GRID_SCALE), np.linspace(ymin, ymax, GRID_SCALE)
    yi, xi = np.meshgrid(yi, xi)
    Xi, Yi, Zi = xi, yi, griddata((inty, intx), intdatavector, (yi, xi), method='cubic')

    mask = (np.sqrt(Xi**2 + Yi**2) <= rmax)
    Zi[~mask] = np.nan

    if noplot == 'off':
        plt.imshow(Zi, extent=(xmin, xmax, ymin, ymax), origin='lower', cmap=cmap)
        plt.colorbar()
        plt.scatter(x, y, c='k')
        plt.plot(np.cos(np.linspace(0, 2 * np.pi, 100)) * rmax, np.sin(np.linspace(0, 2 * np.pi, 100)) * rmax, 'k')
        for i, txt in enumerate(labels):
            plt.annotate(txt, (x[i], y[i]), fontsize=8, ha='right')
        plt.title('Topoplot')
        plt.show()

    return handle, Zi, plotrad, Xi, Yi

In [171]:
import scipy.io
import numpy as np

def load_matlab_file(file_path):
    # Load MATLAB file
    mat_data = scipy.io.loadmat(file_path, struct_as_record=False, squeeze_me=True)
    
    def check_keys(dict_data):
        """
        Check if entries in dictionary are mat-objects. If yes,
        _to_dict is called to change them to dictionaries.
        Recursively go through the entire structure.
        """
        for key in dict_data:
            if isinstance(dict_data[key], scipy.io.matlab.mat_struct):
                dict_data[key] = to_dict(dict_data[key])
            elif isinstance(dict_data[key], dict):
                dict_data[key] = check_keys(dict_data[key])
            elif isinstance(dict_data[key], np.ndarray) and dict_data[key].dtype == object:
                dict_data[key] = np.array([check_keys({i: item})[i] if isinstance(item, dict) else item for i, item in enumerate(dict_data[key])], dtype=object)
                if dict_data[key].size == 0:
                    dict_data[key] = None
        return dict_data

    def to_dict(matobj):
        """
        A recursive function which constructs from matobjects nested dictionaries.
        """
        dict_data = {}
        for strg in matobj._fieldnames:
            elem = getattr(matobj, strg)
            if isinstance(elem, scipy.io.matlab.mat_struct):
                dict_data[strg] = to_dict(elem)
            elif isinstance(elem, np.ndarray) and elem.dtype == object:
                dict_data[strg] = np.array([to_dict(sub_elem) if isinstance(sub_elem, scipy.io.matlab.mat_struct) else sub_elem for sub_elem in elem], dtype=object)
                if dict_data[strg].size == 0:
                    dict_data[strg] = None
            else:
                dict_data[strg] = elem
        # check if contains empty arrays
        for key in dict_data:
            if isinstance(dict_data[key], np.ndarray) and dict_data[key].size == 0:
                dict_data[key] = np.array([])
                
        return dict_data

    return check_keys(mat_data)

# Example usage:
eeglab_file_path = '/System/Volumes/Data/data/matlab/eeglab/sample_data/eeglab_data_epochs_ica.set'
EEG = load_matlab_file(eeglab_file_path)
EEG = EEG['EEG']

# Print the loaded data
res1, res2 = topoplot(EEG['icawinv'].transpose()[0], EEG['chanlocs'])
#res2


QhullError: QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull d Qt Qbb Qz Qc Q12
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1405599788  delaunay  Qtriangulate  Qbbound-last  Qz-infinity-point
  Qcoplanar-keep  Q12-allow-wide  _pre-merge  _zero-centrum  Qinterior-keep
  Pgood  _max-width 90  Error-roundoff 1.2e-13  _one-merge 8.7e-13
  Visible-distance 2.5e-13  U-max-coplanar 2.5e-13  Width-outside 5e-13
  _wide-facet 1.5e-12  _maxoutside 1e-12

precision problems (corrected unless 'Q0' or an error)
      2 degenerate hyperplanes recomputed with gaussian elimination
      2 nearly singular or axis-parallel hyperplanes
      2 zero divisors during back substitute
      2 zero divisors during gaussian elimination

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p1(v4):     0     0     0
- p6

In [74]:
import h5py
import numpy as np

# Create some sample data
data = np.random.rand(100, 100)  # 100x100 array of random numbers

# Create an HDF5 file
with h5py.File('sample_data.h5', 'w') as hdf:
    # Create a dataset in the file
    hdf.create_dataset('dataset_name', data=EEG['chanlocs'])

print("HDF5 file created successfully.")

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [99]:
chan_locs = EEG['chanlocs']


# chan_locs = [{'labels': label, 'theta': theta, 'radius': radius} for label, theta, radius in zip(labels, Th, Rd)]


tmpeloc = chan_locs
labels = [loc['labels'] for loc in tmpeloc]
indices = [i for i, loc in enumerate(tmpeloc) if 'theta' in loc]
indices
Th
# Th = np.array([float(tmpeloc[i]['theta']) if i in indices else np.nan for i in range(len(tmpeloc))])
# Rd = np.array([float(tmpeloc[i]['radius']) if i in indices else np.nan for i in range(len(tmpeloc))])
# Th = np.deg2rad(Th)


array([   0.   ,      nan,  -39.947,    0.   ,   39.897,      nan,
        -69.332,  -44.925,   44.925,   69.332,  -90.   ,  -90.   ,
         90.   ,    0.   ,   90.   , -110.668, -135.075,  135.075,
        110.668, -126.087, -140.053,  180.   ,  140.103,  126.133,
       -144.108, -157.539,  180.   ,  157.539,  144.142, -162.074,
        180.   ,  162.074])

In [182]:
chan_locs = EEG['chanlocs']
datavector = EEG['icawinv'].transpose()[0]

# Set default values
noplot = 'on'
plotgrid = 'off'
plotchans =  []
handle = None
Zi = None
chanval = np.nan
rmax = 0.5  # actual head radius
INTERPLIMITS = 'head'
INTSQUARE = 'on'
default_intrad = 1
ELECTRODES = 'on'
MAXDEFAULTSHOWLOCS = 64
intrad =  np.nan
plotrad = np.nan
headrad =  0.5
squeezefac = 1.0
ContourVals = datavector

# Handle additional arguments
chanrad = noplot[0]
chantheta = noplot[1]
noplot = 'on'

# Set colormap
cmap = plt.get_cmap('jet')
cmaplen = cmap.N
GRID_SCALE = 32

if len(datavector) > MAXDEFAULTSHOWLOCS:
    ELECTRODES = 'off'
else:
    ELECTRODES = 'on'

datavector = np.array(datavector).flatten()
ContourVals = np.array(ContourVals).flatten()

# Read the channel location information
tmpeloc = chan_locs
labels = [loc['labels'] for loc in tmpeloc]
indices = [i for i, loc in enumerate(tmpeloc) if 'theta' in loc]
Th = np.array([tmpeloc[i]['theta'] if i in indices and not isinstance(tmpeloc[i]['theta'], np.ndarray) else np.nan for i in range(len(tmpeloc))])
Rd = np.array([tmpeloc[i]['radius'] if i in indices and not isinstance(tmpeloc[i]['radius'], np.ndarray) else np.nan for i in range(len(tmpeloc))])
Th = np.deg2rad(Th)
allchansind = list(range(len(Th)))
plotchans = indices

if np.isnan(plotrad):
    plotrad = min(1.0, max(Rd) * 1.02)
    plotrad = max(plotrad, 0.5)

if np.isnan(intrad):
    default_intrad = 1
    intrad = min(1.0, max(Rd) * 1.02)
else:
    default_intrad = 0
    if plotrad > intrad:
        plotrad = intrad

if len(tmpeloc) == len(datavector) + 1:
    if plotchans and plotchans[-1] == len(tmpeloc):
        plotchans.pop()

if len(datavector) > 1:
    inds = np.union1d(np.where(np.isnan(datavector))[0], np.where(np.isinf(datavector))[0])
    plotchans = list(set(plotchans) - set(inds))

x, y = np.cos(Th) * Rd, np.sin(Th) * Rd
plotchans = np.abs(plotchans)
allchansind = np.array(allchansind)[plotchans]
Th, Rd, x, y = Th[plotchans], Rd[plotchans], x[plotchans], y[plotchans]
labels = np.array(labels)[plotchans]

pltchans = np.where(Rd <= plotrad)[0]
intchans = np.where((x <= intrad) & (y <= intrad) if INTSQUARE == 'on' else (Rd <= intrad))[0]


allx, ally = x.copy(), y.copy()
labels = labels[pltchans]

if datavector is not None and len(datavector) > 1:
    intdatavector = datavector[intchans]
    datavector = datavector[pltchans]
    ContourVals = ContourVals[pltchans]

squeezefac = rmax / plotrad
Rd, intRd = Rd * squeezefac, Rd[intchans] * squeezefac
x, y, intx, inty = x[pltchans] * squeezefac, y[pltchans] * squeezefac, x[intchans] * squeezefac, y[intchans] * squeezefac


default_intrad
if not np.isnan(default_intrad):
    xmin, xmax = min(-rmax, min(intx)), max(rmax, max(intx))
    ymin, ymax = min(-rmax, min(inty)), max(rmax, max(inty))
else:
    xmin, xmax, ymin, ymax = -intrad * squeezefac, intrad * squeezefac, -intrad * squeezefac, intrad * squeezefac
    
xi, yi = np.linspace(xmin, xmax, GRID_SCALE), np.linspace(ymin, ymax, GRID_SCALE)
yi, xi = np.meshgrid(yi, xi)
Xi, Yi, Zi = xi, yi, griddata((inty, intx), intdatavector, (yi, xi)) # method='cubic'

mask = (np.sqrt(Xi**2 + Yi**2) <= rmax)
# Zi[~mask] = np.nan

if noplot == 'off':
    plt.imshow(Zi, extent=(xmin, xmax, ymin, ymax), origin='lower', cmap=cmap)
    plt.colorbar()
    plt.scatter(x, y, c='k')
    plt.plot(np.cos(np.linspace(0, 2 * np.pi, 100)) * rmax, np.sin(np.linspace(0, 2 * np.pi, 100)) * rmax, 'k')
    for i, txt in enumerate(labels):
        plt.annotate(txt, (x[i], y[i]), fontsize=8, ha='right')
    plt.title('Topoplot')
    plt.show()
    
yi

array([[-0.5       , -0.46774194, -0.43548387, ...,  0.43548387,
         0.46774194,  0.5       ],
       [-0.5       , -0.46774194, -0.43548387, ...,  0.43548387,
         0.46774194,  0.5       ],
       [-0.5       , -0.46774194, -0.43548387, ...,  0.43548387,
         0.46774194,  0.5       ],
       ...,
       [-0.5       , -0.46774194, -0.43548387, ...,  0.43548387,
         0.46774194,  0.5       ],
       [-0.5       , -0.46774194, -0.43548387, ...,  0.43548387,
         0.46774194,  0.5       ],
       [-0.5       , -0.46774194, -0.43548387, ...,  0.43548387,
         0.46774194,  0.5       ]])

In [132]:
np.isnan(plotrad)

True

In [ ]:

xi, yi = np.linspace(xmin, xmax, GRID_SCALE), np.linspace(ymin, ymax, GRID_SCALE)
yi, xi = np.meshgrid(yi, xi)
Xi, Yi, Zi = xi, yi, griddata((inty, intx), datavector[intchans], (yi, xi), method='cubic')

mask = (np.sqrt(Xi**2 + Yi**2) <= rmax)
Zi[~mask] = np.nan

if noplot == 'off':
    plt.imshow(Zi, extent=(xmin, xmax, ymin, ymax), origin='lower', cmap=cmap)
    plt.colorbar()
    plt.scatter(x, y, c='k')
    plt.plot(np.cos(np.linspace(0, 2 * np.pi, 100)) * rmax, np.sin(np.linspace(0, 2 * np.pi, 100)) * rmax, 'k')
    for i, txt in enumerate(labels):
        plt.annotate(txt, (x[i], y[i]), fontsize=8, ha='right')
    plt.title('Topoplot')
    plt.show()